In [1]:
import numpy as np
import torch
import lge

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# U(1) group equivariant model

## Initialize the group

In [2]:
u1_group = lge.U1Group()
u1_rep_dim = u1_group.rep_dim
lie_algebra = u1_group.algebra
print(f"U(1) representation dimension: {u1_rep_dim}")
print(f"U(1) Lie algebra dimension: {lie_algebra.lie_alg_dim}")
print(f'Generator of U(1) Lie algebra : {lie_algebra.generator}')
print(f'Identity element of U(1) group: {u1_group.identity}')

U(1) representation dimension: 1
U(1) Lie algebra dimension: 1
Generator of U(1) Lie algebra : 1j
Identity element of U(1) group: tensor([[1.+0.j]])


## Initialize the equivariant neural network

### 2D lattice

In [3]:
L = 4 # Lattice size
dims = [L, L] # Dimensions of the lattice
hidden_sizes = [5, 5] # Hidden layer sizes. Number of output channels (Wilson loops and links) per lattice site.
kernel_size = 2 # Kernel size or range for convolution, which means the maximum size of Wilson loop.
out_channels = 2 # Number of output channels (Wilson loops and links) per lattice site.

u1_gauge_equivariant_model_2d = lge.LgeConvNet(
    dims=dims,
    hidden_sizes=hidden_sizes,
    kernel_size=kernel_size,
    out_channels=out_channels,
    group=u1_group,
    gauge_invariant=False,
    use_norm=True,
    use_act_fn=True,
    threshold=1e-6
).to(device)

Check the model's structure

In [4]:
lge.check_model(u1_gauge_equivariant_model_2d)

LgeConvNet(
  (group): U1Group(
    (algebra): U1LieAlgebra()
  )
  (plaquette_layer): Plaquette(
    (group): U1Group(
      (algebra): U1LieAlgebra()
    )
  )
  (input_conv): LConvBilin()
  (after_input): Sequential(
    (0): LgeReLU()
    (1): TrNorm()
  )
  (hidden_block): Sequential(
    (0): LConvBilin()
    (1): LgeReLU()
    (2): TrNorm()
    (3): LConvBilin()
    (4): LgeReLU()
    (5): TrNorm()
  )
)
Total number of trainable parameters: 1184


### 4D lattice

In [5]:
L = 4 # Lattice size
dims = [L, L, L, L] # Dimensions of the lattice
hidden_sizes = [5, 5] # Hidden layer sizes. Number of output channels (Wilson loops and links) per lattice site.
kernel_size = 2 # Kernel size or range for convolution, which means the maximum size of Wilson loop.
out_channels = 4 # Number of output channels (Wilson loops and links) per lattice site.

u1_gauge_equivariant_model_4d = lge.LgeConvNet(
    dims=dims,
    hidden_sizes=hidden_sizes,
    kernel_size=kernel_size,
    out_channels=out_channels,
    group=u1_group,
    gauge_invariant=False,
    use_norm=True,
    use_act_fn=True,
    threshold=1e-6
).to(device)

Check the model's structure

In [6]:
lge.check_model(u1_gauge_equivariant_model_4d)

LgeConvNet(
  (group): U1Group(
    (algebra): U1LieAlgebra()
  )
  (plaquette_layer): Plaquette(
    (group): U1Group(
      (algebra): U1LieAlgebra()
    )
  )
  (input_conv): LConvBilin()
  (after_input): Sequential(
    (0): LgeReLU()
    (1): TrNorm()
  )
  (hidden_block): Sequential(
    (0): LConvBilin()
    (1): LgeReLU()
    (2): TrNorm()
    (3): LConvBilin()
    (4): LgeReLU()
    (5): TrNorm()
  )
)
Total number of trainable parameters: 1702


## Initialize the configuration

In [7]:
batch_size = 10
dim = len(dims) # Number of dimensions
num_spacial_points = np.prod(dims) # Number of spacial points
sample_size = batch_size * num_spacial_points * dim
link = u1_group.random_element(
    sample_size=sample_size,
    apply_map=True, # Apply the exponential map.
)
print(f"link shape: {link.shape}")
print(f'Check is unitary: {u1_group.is_unitary(link)}')

# Reshape the link to the desired shape. The shape should be (batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim).
link = link.view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
print(f"link shape: {link.shape}")
print(f'Check is unitary: {u1_group.is_unitary(link)}')

link shape: torch.Size([10240, 1, 1])
Check is unitary: True
link shape: torch.Size([10, 256, 4, 1, 1])
Check is unitary: True


Check the type of link

In [8]:
print(f'Link type: {link.type()}')

Link type: torch.cuda.ComplexFloatTensor


The input to the lattice gauge equivariant model should be a real tensor

In [9]:
link_real = u1_group.convert_to_real(link)
print(f'Link real type: {link_real.type()}')
print(f'Link real shape: {link_real.shape}')

Link real type: torch.cuda.FloatTensor
Link real shape: torch.Size([10, 256, 4, 1, 1, 2])


## Apply the gauge transformation

### Global gauge transformation

Choose a random element from the group

In [10]:
global_g = u1_group.random_element().to(device)
print(f'Random element shape: {global_g.shape}')

Random element shape: torch.Size([1, 1, 1])


Left action of the group element on the link.
$g \cdot link$.

In [11]:
global_g_link = u1_group.left_action_on_Cn(global_g, link)
print(f'g_link shape: {global_g_link.shape}')

g_link shape: torch.Size([10, 256, 4, 1, 1])


Because the group is abelian, do the right action of the group element on the $g \cdot link$ should return the original link.
<br>$g \cdot link \cdot g^{-1} = link \cdot g \cdot g^{-1} = link$

In [12]:
global_g_link_g_inv = u1_group.right_action_on_Cn(global_g, global_g_link)
print(torch.allclose(link, global_g_link_g_inv))

True


### Local gauge transformation

Choose random elements from the group for each spacial point.

In [13]:
local_g = u1_group.random_element(sample_size=sample_size, apply_map=True).to(device)
print(f'Random element shape: {local_g.shape}')
local_g = local_g.view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
print(f'Random element shape: {local_g.shape}')

Random element shape: torch.Size([10240, 1, 1])
Random element shape: torch.Size([10, 256, 4, 1, 1])


$g(x) \cdot link(x)$

In [14]:
local_g_link = u1_group.left_action_on_Cn(local_g, link)
print(f'local_g_link shape: {local_g_link.shape}')

local_g_link shape: torch.Size([10, 256, 4, 1, 1])


$g(x) \cdot link(x) \cdot g^{-1}(x) = link(x) \cdot g(x) \cdot g^{-1}(x) = link(x)$

In [15]:
local_g_link_g_inv = u1_group.right_action_on_Cn(local_g, local_g_link)
print(torch.allclose(link, local_g_link_g_inv))

True


## Check gauge equivariance

Because the U(1) group is abelian, thus, when applying the group element to the link, the output should always be the same.

<br> That is, because the group action on operators like gauge field is defined as:

<br> $\mathcal{L}_{g} \cdot \hat{\mathcal{O}} \equiv g \cdot \hat{\mathcal{O}} \cdot g^{-1} $

<br> where $\hat{\mathcal{O}}$ is the operator, $g$ is the group element, and $\mathcal{L}_{g}$ is the left action of the group element on the operator.

<br> And we have already checked that the left action of the group element on the operator should return the same operator for the U(1) group.

<br> $\mathcal{L}_{g} \cdot \hat{\mathcal{O}} = g \cdot \hat{\mathcal{O}} \cdot g^{-1} = \hat{\mathcal{O}} \cdot g \cdot g^{-1} = \hat{\mathcal{O}}$

<br> Therefore, the output of the model should be the same for the original link and the transformed link. The equivariance condition is always satisfied for the U(1) group.

<br> Also, because the representation of the U(1) group is one-dimensional, so we don't need to trace the Wilson loop to check the gauge invariance. The Wilson loop is always gauge invariant for the U(1) group.

<br> $\mathcal{L}_{g(x)} \cdot Model\left(link(x)\right) =  Model\left(\mathcal{L}_{g(x)} \cdot link(x)\right) = Model\left(link(x)\right)$

## Check the global phase invariant

The Wilson loop is always invariant under a global phase transformation.

In [16]:
link = u1_group.random_element(sample_size=sample_size).view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
link_real = u1_group.convert_to_real(link)
global_g = u1_group.random_element().to(device)
global_g_link = u1_group.left_action_on_Cn(global_g, link)

Convert the link to real type

In [17]:
global_g_link = u1_group.convert_to_real(global_g_link)
print(f'Global g*link real type: {global_g_link.type()}')
print(f'Global g*link real shape: {global_g_link.shape}')

Global g*link real type: torch.cuda.FloatTensor
Global g*link real shape: torch.Size([10, 256, 4, 1, 1, 2])


Apply the model to the link and the transformed link.
<br>$Model \left(g \cdot link \right)$
<br>$Model \left( link \right)$

In [18]:
model_global_g_link = u1_gauge_equivariant_model_4d(global_g_link)
print(f'Model(g*link) shape: {model_global_g_link.shape}')

model_link = u1_gauge_equivariant_model_4d(link_real)
print(f'Model(link) shape: {model_link.shape}')

Model(g*link) shape: torch.Size([10, 256, 4, 1, 1, 2])
Model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])


Convert the output to complex type

In [19]:
model_global_g_link = u1_group.convert_to_complex(model_global_g_link)
print(f'Model(g*link) shape: {model_global_g_link.shape}')

model_link = u1_group.convert_to_complex(model_link)
print(f'Model(link) shape: {model_link.shape}')

Model(g*link) shape: torch.Size([10, 256, 4, 1, 1])
Model(link) shape: torch.Size([10, 256, 4, 1, 1])


Ensure the invariant condition is satisfied
<br> $Model \left( link \right) = Model \left( g \cdot link \right)$

In [20]:
print(torch.allclose(model_link, model_global_g_link, atol=1e-3))

True


Local phase transformation

In [21]:
link = u1_group.random_element(sample_size=sample_size).view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
link_real = u1_group.convert_to_real(link)
local_g = u1_group.random_element(sample_size=sample_size).view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim).to(device)
local_g_link = u1_group.left_action_on_Cn(local_g, link)

Convert the link to real type

In [22]:
local_g_link = u1_group.convert_to_real(local_g_link)
print(f'Local g(x)*link real type: {local_g_link.type()}')
print(f'Local g(x)*link real shape: {local_g_link.shape}')

Local g(x)*link real type: torch.cuda.FloatTensor
Local g(x)*link real shape: torch.Size([10, 256, 4, 1, 1, 2])


<br>$Model \left(g(x) \cdot link(x) \right)$
<br>$Model \left(link(x) \right)$

In [23]:
model_local_g_link = u1_gauge_equivariant_model_4d(local_g_link)
print(f'Model(g(x)*link(x)) shape: {model_local_g_link.shape}')

model_link = u1_gauge_equivariant_model_4d(link_real)
print(f'Model(link) shape: {model_link.shape}')

Model(g(x)*link(x)) shape: torch.Size([10, 256, 4, 1, 1, 2])
Model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])


Convert the output to complex type

In [24]:
model_local_g_link = u1_group.convert_to_complex(model_local_g_link)
print(f'Model(g(x)*link(x)) shape: {model_local_g_link.shape}')

model_link = u1_group.convert_to_complex(model_link)
print(f'Model(link) shape: {model_link.shape}')

Model(g(x)*link(x)) shape: torch.Size([10, 256, 4, 1, 1])
Model(link) shape: torch.Size([10, 256, 4, 1, 1])


Usually not invariant.
<br> $Model \left( link(x) \right) \neq Model \left( g(x) \cdot link(x) \right)$

In [25]:
print(torch.allclose(model_link, model_local_g_link, atol=1e-5))

False
